In [3]:
# 기본
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid') # sns에 흰색 그리드 유지
import missingno # 결측치 시각화

# KFold (교차 검증을 사용하기 위해)
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate

# 학습 데이터와 검증 데이터로 나누는 함수
from sklearn.model_selection import train_test_split

# 데이터 전처리
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 하이퍼 파라미터 튜닝
from sklearn.model_selection import GridSearchCV

# 평가 함수
from sklearn.metrics import accuracy_score

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import  XGBRegressor

# 머신러닝 알고리즘 - 군집
from sklearn.cluster import KMeans
from sklearn.cluster import MeanShift

# 머신러닝 알고리즘 - 차원축소
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# 다중분류를 위한 원핫 인코더
from keras.utils import to_categorical

# 학습 자동 중단
from keras.callbacks import EarlyStopping

# 모델 저장
from keras.callbacks import ModelCheckpoint

# 저장된 딥러닝 모델 불러오기
from keras.models import load_model

# 딥러닝
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import LeakyReLU
from keras.layers import BatchNormalization
from keras.layers import Reshape
from keras.layers import UpSampling2D
from keras.layers import Input
from keras.models import Model

# 이미지 생성자
from keras.preprocessing.image import ImageDataGenerator

# VGG16 모델 (이미 학습이 완료되어 있는 이미지 인식 모델)
from keras.applications import VGG16

# 자연어 처리
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

# 전처리, 활성 함수 셋팅
from keras import optimizers, initializers, regularizers, metrics

# 저장
import pickle

# 시간 모듈
import time

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'   # 윈도우용
# plt.rcParams['font.family'] = 'AppleGothic'   # 맥용
plt.rcParams['font.size'] = 10                 # 폰트 크기
plt.rcParams['figure.figsize'] = 15,8          # 그래프 크기
plt.rcParams['axes.unicode_minus'] = False     # - 기호 깨짐 방지


# 경고 메시지가 안나오게 하기
import warnings
warnings.filterwarnings('ignore')

# gpu 사용 초기화 및 할당
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
# seed 설정
np.random.seed(3)
tf.random.set_seed(3)

In [7]:
# 폴더 목록을 가져온다.
categories= list(os.walk('../images/101_ObjectCategories/'))[0][1]

In [12]:
# 이미지의 크기값
image_w = 64
image_h = 64

In [13]:
# 전처리 후 저장한 데이터를 읽어온다.
X_train, X_test, y_train, y_test = np.load('../images/caltech_obj_101.npy',
                                          allow_pickle=True)

In [14]:
# 데이터 정규화 (색상값을 0 ~ 1 사이로 조정)
X_train = X_train.astype('float32') / 256
X_test = X_test.astype('float32') / 256
print(X_train.shape)
print(X_test.shape)

(6858, 64, 64, 3)
(2286, 64, 64, 3)


In [17]:
# 모델 구축하기
model = Sequential()

model.add(Conv2D(32, kernel_size=(3,3), input_shape=X_train.shape[1:],
                padding = 'same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=4))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3,3),padding = 'same'))
model.add(Activation('relu'))

model.add(Conv2D(64, kernel_size=(3,3)))
model.add(MaxPooling2D(pool_size=4))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(len(categories)))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_6 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 14, 64)       

In [28]:
# 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# 모델 저장 조건
model_path = 'models/Caltech/{epoch}-{val_loss}.hdf5'
callback1 = ModelCheckpoint(filepath=model_path, monitor='val_loss',
                           verbose=1, save_best_only=True)

In [30]:
# 자동 중단 설정
callback2 = EarlyStopping(monitor='val_loss', patience=100)

In [31]:
# 학습
model.fit(X_train, y_train, validation_data=(X_test, y_test),
         batch_size=32, epochs=1000, callbacks=[callback1, callback2])

Epoch 1/1000
215/215 [==============================] - 2s 7ms/step - loss: 0.1285 - accuracy: 0.9608 - val_loss: 2.4391 - val_accuracy: 0.6907

Epoch 00001: val_loss improved from inf to 2.43906, saving model to models/Caltech\1-2.4390642642974854.hdf5
Epoch 2/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1527 - accuracy: 0.9525 - val_loss: 2.3783 - val_accuracy: 0.6881

Epoch 00002: val_loss improved from 2.43906 to 2.37832, saving model to models/Caltech\2-2.378319501876831.hdf5
Epoch 3/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1482 - accuracy: 0.9603 - val_loss: 2.6702 - val_accuracy: 0.6772

Epoch 00003: val_loss did not improve from 2.37832
Epoch 4/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1387 - accuracy: 0.9561 - val_loss: 2.5831 - val_accuracy: 0.6763

Epoch 00004: val_loss did not improve from 2.37832
Epoch 5/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1544 - accuracy: 0.

215/215 [==============================] - 1s 6ms/step - loss: 0.1503 - accuracy: 0.9568 - val_loss: 2.6938 - val_accuracy: 0.6833

Epoch 00042: val_loss did not improve from 2.37832
Epoch 43/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1883 - accuracy: 0.9534 - val_loss: 2.9129 - val_accuracy: 0.6815

Epoch 00043: val_loss did not improve from 2.37832
Epoch 44/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1931 - accuracy: 0.9508 - val_loss: 2.7261 - val_accuracy: 0.6868

Epoch 00044: val_loss did not improve from 2.37832
Epoch 45/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1481 - accuracy: 0.9577 - val_loss: 3.0896 - val_accuracy: 0.6864

Epoch 00045: val_loss did not improve from 2.37832
Epoch 46/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1640 - accuracy: 0.9564 - val_loss: 2.8403 - val_accuracy: 0.6864

Epoch 00046: val_loss did not improve from 2.37832
Epoch 47/1000
215/215 [======


Epoch 00083: val_loss did not improve from 2.37832
Epoch 84/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1800 - accuracy: 0.9540 - val_loss: 3.0470 - val_accuracy: 0.6789

Epoch 00084: val_loss did not improve from 2.37832
Epoch 85/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1713 - accuracy: 0.9586 - val_loss: 2.9198 - val_accuracy: 0.6649

Epoch 00085: val_loss did not improve from 2.37832
Epoch 86/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1833 - accuracy: 0.9505 - val_loss: 2.9331 - val_accuracy: 0.6737

Epoch 00086: val_loss did not improve from 2.37832
Epoch 87/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1719 - accuracy: 0.9510 - val_loss: 3.1781 - val_accuracy: 0.6846

Epoch 00087: val_loss did not improve from 2.37832
Epoch 88/1000
215/215 [==============================] - 1s 6ms/step - loss: 0.1473 - accuracy: 0.9600 - val_loss: 3.0084 - val_accuracy: 0.6789

Epoch 00088: v

In [32]:
# 베스트 모델
best_model = load_model('models/Caltech/2-2.378319501876831.hdf5')

In [33]:
score = best_model.evaluate(X_test, y_test)
print(f'손실 : {score[0]}')
print(f'정확도 : {score[1]}')

72/72 [==============================] - 0s 3ms/step - loss: 2.3783 - accuracy: 0.6881
손실 : 2.378319501876831
정확도 : 0.6881014704704285
